In [1]:
import lumipy as lm
from lusidjam import RefreshingToken as rt

atlas = lm.get_atlas(token=rt())

In [2]:
# Get the example portfolios from Finbourne-Examples
all_portfolios = atlas.lusid_portfolio_holding()

example_pfs = all_portfolios.select('*').where(
    all_portfolios.portfolio_scope == 'Finbourne-Examples'
).to_table_var()

# Tutorial 5 - Joins in Fluent Syntax

## Introduction

All table classes in lumipy have a collection of join methods such as `inner_join` and `left_join`. These can take any source table class except for another join.

The join expressions are flexible enough to accept columns/expressions from any of their parent tables and automatically handle prefixing and aliasing in the case of clashing column names. Table aliases can be specified in the join method (`right_alias`/`left_alias` args) or by supplying aliased tables (see cell below).

In [3]:
portfolios = example_pfs.with_alias('ptf')
holdings = atlas.lusid_portfolio_holding().with_alias('hld')
instruments = atlas.lusid_instrument().with_alias('ins')
properties = atlas.lusid_property().with_alias('prp')
quotes = atlas.lusid_instrument_quote().with_alias('qte')

## Simple Join
In this example we'll join example portfolios and holdings on portfolio code with the additional condition that holdings are in the scopre Finbourne-Examples (this speeds the query up).

Note that the `on` argument will take any expression that is made out of columns from the parent tables and resolves to a boolean. 

In [4]:
join = example_pfs.left_join(
    holdings,
    on=(holdings.portfolio_code == example_pfs.portfolio_code) &
       (holdings.portfolio_scope == 'Finbourne-Examples')
)

qry = join.select('^')

In [5]:
df = qry.go()
df.head()

,CostAmount_lhs,CostCurrency_lhs,Error_lhs,HoldingType_lhs,LusidInstrumentId_lhs,PortfolioCode_lhs,PortfolioScope_lhs,SettledUnits_lhs,Units_lhs,CostAmount_hld,CostCurrency_hld,Error_hld,HoldingType_hld,LusidInstrumentId_hld,PortfolioCode_hld,PortfolioScope_hld,SettledUnits_hld,Units_hld
0,2.390760e+09,USD,NaN,Position,LUID_JTQY6QFI,Global-Equity,Finbourne-Examples,12000000.0,12000000.0,-1.092849e+09,USD,NaN,Position,LUID_WW55WKWV,Global-Equity,Finbourne-Examples,-8.191471e+06,-8.191471e+06
1,2.390760e+09,USD,NaN,Position,LUID_JTQY6QFI,Global-Equity,Finbourne-Examples,12000000.0,12000000.0,-3.584603e+07,JPY,NaN,Balance,CCY_JPY,Global-Equity,Finbourne-Examples,-3.584603e+07,-3.584603e+07
2,2.390760e+09,USD,NaN,Position,LUID_JTQY6QFI,Global-Equity,Finbourne-Examples,12000000.0,12000000.0,-2.734676e+07,GBP,NaN,Balance,CCY_GBP,Global-Equity,Finbourne-Examples,-2.734676e+07,-2.734676e+07
3,2.390760e+09,USD,NaN,Position,LUID_JTQY6QFI,Global-Equity,Finbourne-Examples,12000000.0,12000000.0,-2.450548e+07,USD,NaN,Balance,CCY_USD,Global-Equity,Finbourne-Examples,-2.450548e+07,-2.450548e+07
4,2.390760e+09,USD,NaN,Position,LUID_JTQY6QFI,Global-Equity,Finbourne-Examples,12000000.0,12000000.0,3.109738e+08,GBP,NaN,Position,LUID_YU1CI1C0,Global-Equity,Finbourne-Examples,4.741176e+05,4.741176e+05


## Chaining Joins Together

You can build up more complex joins with more than two tables by chaining join methods together. 

In [6]:
join = example_pfs.left_join(
    holdings,
    on=(example_pfs.portfolio_code == holdings.portfolio_code) &
       (holdings.portfolio_scope == 'Finbourne-Examples')
).left_join(
    properties,
    on=(properties.domain == 'Instrument') &
       (properties.entity_id_type == 'LusidInstrumentId') &
       (properties.entity_id == holdings.lusid_instrument_id)
).left_join(
    quotes,
    on=holdings.lusid_instrument_id == quotes.instrument_id
)

## Filtering and Aggregating Joins

Once you have a join table built you can then call select() and chain the other query methods as usual. 

In [9]:
qry = join.select(
    holdings.portfolio_code,
    holdings.lusid_instrument_id
).where(
    join.value_qte.is_not_null()
).group_by(
    holdings.lusid_instrument_id
).aggregate(
    MeanQuoteValue=join.value_qte.mean(),
    StddQuoteValue=join.value_qte.stdev()   
)
df = qry.go()
df

,PortfolioCode,LusidInstrumentId,MeanQuoteValue,StddQuoteValue
